In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain_google_genai sqlite3 langgraph-checkpoint-sqlite

In [ ]:
## in memory database

import sqlite3

conn = sqlite3.connect(":memory:")

In [ ]:
# saved to local db

db_path = "state_db/example.db"

conn = sqlite3.connect(db_path, check_same_thread=False)

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver(conn)

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 

In [ ]:
from langgraph.graph import MessagesState

class State(MessagesState):
    summary: str

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

def call_model(state: State):
    
    summary = state.get("summary", "")
    
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
        
    response = llm.invoke(messages)
    return {"messages": response}

In [ ]:
def summarize_conversation(state: State):
    
    summary = state.get("summary", "")
    
    if summary:
        summary_message = """
            This is the summary of the conversation to date: {summary}
            Extend the summary by taken into account the new messages above:
        """
    else:
        summary_message = "Create a summary of the conversation above"
        
    
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)
    
    delete_messages = [RemoveMessage(id = m.id) for m in state["messages"][:-2]]
    
    return {
        "summary": response.content,
        "messages": delete_messages
    }

In [ ]:
from langgraph.graph import END
from typing_extensions import Literal

def should_continue(state: State)-> Literal["summarize_conversation", END]:
    """Return the next node to decide"""
    
    messages = state["messages"]
    
    if len(messages) > 6:
        return "summarize_conversation"
    
    return END

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START
#from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

#memory = MemorySaver()

graph = workflow.compile(checkpointer=memory)

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def print_message(messages):
    for m in messages[-1:]:
        m.pretty_print()

config = {
    "configurable": {
        "thread_id": "1"
    }
}





In [ ]:
input_message = HumanMessage(content="Hi, I am Muhammad")
output = graph.invoke({
    "messages": [input_message]
    }, config
) 
print_message(output["messages"])


input_message = HumanMessage(content="What is my Name")
output = graph.invoke({
    "messages": [input_message]
    }, config
) 
print_message(output["messages"])

input_message = HumanMessage(content="I like Langgraph")
output = graph.invoke({
    "messages": [input_message]
    }, config
) 
print_message(output["messages"])


In [ ]:
graph.get_state(config).values.get("summary", "")

In [ ]:
input_message = HumanMessage(content="I like A2A and MCP")
output = graph.invoke({
    "messages": [input_message]
    }, config
) 
print_message(output["messages"])

In [ ]:
graph.get_state(config).values.get("summary", "")

In [ ]:
graph.get_state(config)